In [30]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import itertools
import scipy.stats as st
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.feature_selection import mutual_info_classif
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
import rpy2
#import pingouin as pg
from itertools import combinations

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.rinterface_lib.embedded import RRuntimeError

# import R's "utils" package
utils = importr('utils')
lavaan = importr('lavaan')
semtools = importr('semTools')
stringr = importr('stringr')
reticulate = importr('reticulate')
#lavaanPlot = importr('lavaanPlot')

In [31]:
# importing stuff the right way
try:
    semTools_version = '.'.join([str(ii) for ii in list(ro.r('packageVersion("semTools")')[0])])
except RRuntimeError:
    semTools_version = ''
# There's a bug in the main semtools libarary, this install from my fork of semtools where I've fixed it.
if semTools_version != '0.5.6.933':
    ro.r("""
        library(devtools)
        install_github("shotgunosine/semTools/semTools@aa68e38f88d9dd881617d5ade429bde2b2a74e08")
        """
    )

In [62]:
# general
#data_path = '../data/mydata_1general_forCFA_removedchecks.csv'
#path_to_save = '../results/CFA_general_removedchecks.csv'

data_path = '../data/mydata_1general_forCFA_removedGRIDchecks.csv'
path_to_save = '../results/CFA_general_removedGRIDchecks.csv'

#data_path = '../data/mydata_1general_forCFA.csv'
#path_to_save = '../results/CFA_general.csv'

#highrisk
#data_path = '../data/mydata_1highrisk_forCFA_removedchecks.csv'
#path_to_save = '../results/CFA_highrisk_removedchecks.csv'

#data_path = '../data/mydata_1highrisk_forCFA_removedGRIDchecks.csv'
#path_to_save = '../results/CFA_highrisk_removedGRIDchecks.csv'

#data_path = '../data/mydata_1highrisk_forCFA.csv'
#path_to_save = '../results/CFA_highrisk.csv'

mydata = pd.read_csv(data_path)
mydata = mydata.rename(columns={'Unnamed: 0': 'Subject'})
mydata.head()

,Subject,hitop_sum,hitop_sum_recontact,baars_sum,baars_sum_recontact,phq_sum,phq_sum_recontact,gad_sum,gad_sum_recontact,mood_bothered,mood_bothered_recontact,anxiety_bothered,anxiety_bothered_recontact,attention_bothered,attention_bothered_recontact,moodanxiety_bothered,moodanxiety_bothered_recontact,inattention_sum,inattention_sum_recontact,hyperactivity_sum,hyperactivity_sum_recontact,impulsivity_sum,impulsivity_sum_recontact,sct_sum,sct_sum_recontact,anhedonic_depression,anhedonic_depression_recontact,anxious_worry,anxious_worry_recontact,appetite_gain,appetite_gain_recontact,appetite_loss,appetite_loss_recontact,cognitive_problems,cognitive_problems_recontact,hyposomnia,hyposomnia_recontact,indecisiveness,indecisiveness_recontact,insomnia,insomnia_recontact,panic,panic_recontact,separation_insecurity,separation_insecurity_recontact,shame_guilt,shame_guilt_recontact,situational_phobia,situational_phobia_recontact,social_anxiety,social_anxiety_recontact,well_being,well_being_recontact,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,inattention_1_recontact,inattention_2_recontact,inattention_3_recontact,inattention_4_recontact,inattention_5_recontact,inattention_6_recontact,inattention_7_recontact,inattention_8_recontact,inattention_9_recontact,hyperactivity_1_recontact,hyperactivity_2_recontact,hyperactivity_3_recontact,hyperactivity_4_recontact,hyperactivity_5_recontact,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact
0,0,39,22,16,11,7,3,10,6,False,False,False,False,False,False,False,False,7,4,7,7,2,0,15,9,2,3,11,6,5,4,0,0,6,2,0,0,3,1,1,2,0,0,4,2,2,0,0,0,5,2,28,26,0,1,0,0,0,1,0,3,2,1,0,1,2,3,0,2,0,0,0,0,0,1,1,0,0,1,1,3,0,2,1,1,0,0,0,0,2,2,2,2,1,1,0,0,1,0,3,0,2,1,0,0,1,2,1,3,2,3,3,0,1,1,1,1,2,0,0,0,0,0,1,0,1,1,0,1,0,3,1,3,0,0,0,1
1,1,93,77,12,11,4,6,6,4,False,False,False,False,False,False,False,False,6,5,3,4,3,2,11,9,10,10,13,9,5,4,2,4,6,4,5,6,3,4,4,8,12,5,3,0,3,0,10,10,17,13,27,26,1,1,1,0,0,1,0,1,1,0,0,1,1,1,1,1,0,1,0,0,1,0,0,0,1,2,1,1,0,1,1,1,0,1,0,1,1,1,1,1,1,0,1,0,0,1,0,1,0,1,1,2,2,1,2,1,1,1,1,0,2,1,0,0,1,0,0,1,0,2,1,1,0,1,0,2,1,1,2,1,1,0,0,1
2,3,17,29,4,3,0,0,0,0,False,False,False,False,False,False,False,False,4,2,0,1,0,0,3,1,1,4,0,5,3,0,0,6,1,1,0,0,0,1,6,4,1,3,0,1,0,1,4,3,1,0,13,11,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,92,77,16,18,13,9,4,7,False,False,False,False,False,False,False,False,8,10,3,4,5,4,11,11,16,15,6,10,12,7,0,4,9,8,2,2,3,4,10,7,9,5,8,4,4,4,2,0,11,7,8,4,1,1,1,1,0,0,1,1,2,2,0,1,0,0,0,2,2,1,1,2,2,1,0,0,1,2,1,2,0,1,0,1,1,1,1,1,0,1,1,1,1,0,0,3,0,2,2,3,0,3,0,1,1,1,1,2,2,2,1,0,1,1,1,1,1,1,1,2,2,1,1,1,1,1,0,1,0,1,1,2,2,2,1,1
4,5,65,48,17,14,10,6,8,5,False,False,True,True,False,False,True,True,9,9,6,3,2,2,8,8,10,8,9,7,4,4,3,0,7,4,0,0,3,3,0,0,2,3,4,1,4,2,3,1,16,15,15,13,1,1,0,0,1,2,1,2,1,3,0,2,1,0,1,0,0,1,1,1,0,0,1,2,1,2,1,1,0,1,1,0,1,0,0,1,1,1,1,1,1,2,1,1,1,3,1,1,1,1,1,1,1,0,2,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,0,1,0,2,1,0,2,0,1,1,1,0


In [39]:
def prep_data_recontactanalysis(dataframe_withrecontact):
    df = dataframe_withrecontact
    df_helper = df.copy(deep=True)
    df_rows = df.shape[0]
    df = df.assign(whichvisit=['initialvisit']*df_rows)
    df_helper = df_helper.assign(whichvisit=['recontactvisit']*df_rows)
    # in df, drop all "recontact" columns
    # in df_helper, drop all initial visit columns, then rename the recontact columns
    cols_recontact = []
    cols_original = []
    for c in df.columns:
        if "recontact" in c:
            cols_recontact.append(c)
            cols_original.append(c[:-10])  
    df = df.drop(columns=cols_recontact)
    df_helper = df_helper.drop(columns=cols_original)
    cols_dict = {}
    for cname in cols_recontact:
        cols_dict[cname] = cname[:-10]
    # rename cols in df_helper
    df_helper.rename(columns=cols_dict, inplace=True)    
    mydata_withrecontact = pd.concat([df, df_helper])
    return(mydata_withrecontact)

mydata_forcfa = prep_data_recontactanalysis(mydata)
mydata_forcfa.head(100)

,Subject,hitop_sum,baars_sum,phq_sum,gad_sum,mood_bothered,anxiety_bothered,attention_bothered,moodanxiety_bothered,inattention_sum,hyperactivity_sum,impulsivity_sum,sct_sum,anhedonic_depression,anxious_worry,appetite_gain,appetite_loss,cognitive_problems,hyposomnia,indecisiveness,insomnia,panic,separation_insecurity,shame_guilt,situational_phobia,social_anxiety,well_being,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,whichvisit
0,0,80,13,14,7,True,False,False,True,6,6,1,7,11,6,9,0,1,0,0,12,3,10,9,7,12,11,0,0,0,0,1,0,3,1,1,2,0,1,0,3,1,0,0,0,1,0,0,3,1,2,0,1,1,3,3,3,2,1,0,0,2,0,2,0,2,1,0,0,initialvisit
1,2,123,41,15,14,True,True,True,True,21,11,9,25,16,17,2,2,12,9,9,5,8,12,10,2,19,8,3,3,2,3,1,1,3,3,2,1,3,3,1,3,2,2,3,2,2,2,2,2,2,3,1,2,2,2,2,1,2,3,1,3,3,3,3,3,3,3,1,3,initialvisit
2,3,84,0,12,16,False,False,False,False,0,0,0,12,12,10,3,2,4,0,4,12,12,4,9,0,12,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,3,1,2,3,1,2,3,3,2,0,1,0,1,1,1,1,1,3,3,0,1,initialvisit
3,5,156,15,21,21,True,True,True,True,12,3,0,20,27,20,2,6,10,0,9,12,12,20,11,1,26,6,1,1,1,1,1,2,1,2,2,1,0,1,1,0,0,0,0,0,3,3,3,3,3,3,3,2,3,3,3,3,3,3,1,3,3,2,0,3,3,1,2,3,initialvisit
4,8,96,8,7,15,False,True,False,True,5,1,2,11,8,18,2,0,5,0,5,3,5,15,6,1,28,18,0,1,0,1,1,0,0,1,1,1,0,0,0,0,0,2,0,0,3,3,3,3,0,0,3,0,1,2,2,0,2,0,0,1,0,0,0,0,2,3,2,3,initialvisit
5,9,52,9,14,3,False,True,True,True,9,0,0,9,18,4,8,0,5,0,4,0,0,3,6,0,4,5,1,1,1,2,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,3,2,3,3,1,0,1,0,0,0,1,2,2,1,2,initialvisit
6,10,132,9,14,12,True,True,False,True,4,2,3,6,26,18,12,0,4,6,2,1,11,8,11,6,27,7,1,0,1,0,0,1,1,0,0,1,0,0,0,1,0,2,0,1,2,2,2,1,1,2,2,1,1,0,3,3,3,3,0,0,0,1,0,0,1,2,1,1,initialvisit
7,11,108,24,17,11,True,True,True,True,12,5,7,15,11,16,8,5,5,7,6,7,8,13,4,3,15,17,2,1,1,2,1,1,1,2,1,1,1,1,1,1,1,1,3,2,2,1,1,1,1,3,2,2,3,3,1,2,2,2,2,2,1,0,3,3,3,1,1,1,initialvisit
8,12,83,30,14,15,True,True,False,True,18,6,6,13,11,14,11,0,10,0,7,8,5,5,6,1,5,11,2,3,1,2,2,2,2,3,1,2,0,1,2,1,1,2,2,1,2,3,2,3,2,1,2,1,1,3,2,3,1,2,1,3,1,1,1,2,2,2,1,0,initialvisit
9,13,144,26,20,18,True,True,False,True,15,4,7,19,27,15,3,7,9,9,5,12,15,11,11,0,20,0,2,3,1,2,0,1,3,1,2,0,1,0,3,0,3,3,0,1,3,3,3,3,0,3,3,3,3,3,3,2,2,3,1,0,2,3,3,3,3,3,1,1,initialvisit


In [40]:
def print_summary(obj):
    return rprint(summary(obj))

def print_short_summary(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r("print(myoutput[stringid-1])")
    ro.r("print(myoutput[stringid])")
    return (0)

def print_problematic_items(invariance_output):
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    print('Problematic items:')
    ro.r("stringidsig <- grep(\"may differ between Groups\", myoutput)")
    ro.r("print(myoutput[stringidsig])")   
    return (0)
    
def extract_p(invariance_output):
    ro.globalenv['invariance_output'] = invariance_output
    ro.r("myoutput <- capture.output(summary(invariance_output))") 
    ro.r("stringid <- grep(\"chisq\", myoutput)")
    ro.r('myrline <- myoutput[stringid]')
    mypline = ro.r('myrline')[0]
    my_p = mypline.split()[-1]
    return(my_p)

def check_secondary_criteria(fit_config):
    Criteria_passed = False
    ro.r("myoutputconfig <- capture.output(summary(fit_config, fit.measures=TRUE))")
    ro.r("stringid_cfi <- grep(\"Robust.*CFI\", myoutputconfig)")
    ro.r("cfi_line <- myoutputconfig[stringid_cfi]")
    cfiline = ro.r('cfi_line')[0]    
    my_cfi = cfiline.split()[-1] 
    my_cfi = float(my_cfi)
    ro.r("stringid_tli <- grep(\"Robust.*TLI\", myoutputconfig)")
    ro.r("tli_line <- myoutputconfig[stringid_tli]")
    tliline = ro.r('tli_line')[0]    
    my_tli = tliline.split()[-1] 
    my_tli = float(my_tli)
    ro.r("stringid_rmsea <- grep(\"Robust.*RMSEA\", myoutputconfig)")
    ro.r("rmsea_line <- myoutputconfig[stringid_rmsea]")
    myrmsealine = ro.r('rmsea_line')[0]
    my_rmsea = myrmsealine.split('\n')[0].split()[-1]
    my_rmsea = float(my_rmsea)   
    if (my_cfi > 0.95) and (my_tli > 0.95) and (my_rmsea < 0.06):
        Criteria_passed = True
    return (Criteria_passed, my_cfi, my_tli, my_rmsea)

In [41]:
for c in mydata_forcfa.columns:
    print(c)

Subject
hitop_sum
baars_sum
phq_sum
gad_sum
mood_bothered
anxiety_bothered
attention_bothered
moodanxiety_bothered
inattention_sum
hyperactivity_sum
impulsivity_sum
sct_sum
anhedonic_depression
anxious_worry
appetite_gain
appetite_loss
cognitive_problems
hyposomnia
indecisiveness
insomnia
panic
separation_insecurity
shame_guilt
situational_phobia
social_anxiety
well_being
inattention_1
inattention_2
inattention_3
inattention_4
inattention_5
inattention_6
inattention_7
inattention_8
inattention_9
hyperactivity_1
hyperactivity_2
hyperactivity_3
hyperactivity_4
hyperactivity_5
impulsivity_1
impulsivity_2
impulsivity_3
impulsivity_4
gad_1
gad_2
gad_3
gad_4
gad_5
gad_6
gad_7
phq_1
phq_2
phq_3
phq_4
phq_5
phq_6
phq_7
phq_8
sct_1
sct_2
sct_3
sct_4
sct_5
sct_6
sct_7
sct_8
sct_9
whichvisit


In [42]:
mydata_forcfa.head()

,Subject,hitop_sum,baars_sum,phq_sum,gad_sum,mood_bothered,anxiety_bothered,attention_bothered,moodanxiety_bothered,inattention_sum,hyperactivity_sum,impulsivity_sum,sct_sum,anhedonic_depression,anxious_worry,appetite_gain,appetite_loss,cognitive_problems,hyposomnia,indecisiveness,insomnia,panic,separation_insecurity,shame_guilt,situational_phobia,social_anxiety,well_being,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,whichvisit
0,0,80,13,14,7,True,False,False,True,6,6,1,7,11,6,9,0,1,0,0,12,3,10,9,7,12,11,0,0,0,0,1,0,3,1,1,2,0,1,0,3,1,0,0,0,1,0,0,3,1,2,0,1,1,3,3,3,2,1,0,0,2,0,2,0,2,1,0,0,initialvisit
1,2,123,41,15,14,True,True,True,True,21,11,9,25,16,17,2,2,12,9,9,5,8,12,10,2,19,8,3,3,2,3,1,1,3,3,2,1,3,3,1,3,2,2,3,2,2,2,2,2,2,3,1,2,2,2,2,1,2,3,1,3,3,3,3,3,3,3,1,3,initialvisit
2,3,84,0,12,16,False,False,False,False,0,0,0,12,12,10,3,2,4,0,4,12,12,4,9,0,12,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,3,1,2,3,1,2,3,3,2,0,1,0,1,1,1,1,1,3,3,0,1,initialvisit
3,5,156,15,21,21,True,True,True,True,12,3,0,20,27,20,2,6,10,0,9,12,12,20,11,1,26,6,1,1,1,1,1,2,1,2,2,1,0,1,1,0,0,0,0,0,3,3,3,3,3,3,3,2,3,3,3,3,3,3,1,3,3,2,0,3,3,1,2,3,initialvisit
4,8,96,8,7,15,False,True,False,True,5,1,2,11,8,18,2,0,5,0,5,3,5,15,6,1,28,18,0,1,0,1,1,0,0,1,1,1,0,0,0,0,0,2,0,0,3,3,3,3,0,0,3,0,1,2,2,0,2,0,0,1,0,0,0,0,2,3,2,3,initialvisit


In [43]:
mydata = mydata_forcfa
whichcolumn = 'enriched_withrecontact'

assert(whichcolumn in ['enriched_withrecontact', 'genpop_withrecontact'])
num_iter = 1000
rprint = ro.r('print')
summary = ro.r('summary')
ro.r('set.seed(12345)')
rngkind = "L'Ecuyer-CMRG"

mdl_strs_new_new = {'hitop_SUM': 'hitop_SUM =~anhedonic_depression + anxious_worry + appetite_gain + appetite_loss + cognitive_problems + hyposomnia + indecisiveness + insomnia + panic + separation_insecurity + shame_guilt + situational_phobia + social_anxiety', 
'baars_SUM': 'baars_SUM =~inattention_sum + hyperactivity_sum + impulsivity_sum + sct_sum',
'phq_SUM': 'phq_SUM =~phq_1 + phq_2 + phq_3 + phq_4 + phq_5 + phq_6 + phq_7 + phq_8',
'gad_SUM': 'gad_SUM =~gad_1 + gad_2 + gad_3 + gad_4 + gad_5 + gad_6 + gad_7',
'baars_inattention_SUM': 'baars_inattention_SUM =~inattention_1 + inattention_2 + inattention_3 + inattention_4 + inattention_5 + inattention_6 + inattention_7 + inattention_8 + inattention_9',
'baars_hyperactivity_SUM': 'baars_hyperactivity_SUM =~hyperactivity_1 + hyperactivity_2 + hyperactivity_3 + hyperactivity_4 + hyperactivity_5',
'baars_impulsivity_SUM': 'baars_impulsivity_SUM =~impulsivity_1 + impulsivity_2 + impulsivity_3 + impulsivity_4',
'baars_sct_SUM': 'baars_sct_SUM =~sct_1 + sct_2 + sct_3 + sct_4 + sct_5 + sct_6 + sct_7 + sct_8 + sct_9'}

if whichcolumn == 'enriched_withrecontact':
    mydata.to_csv('../data/CFA/data_enriched.csv')
    ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
    group = 'whichvisit'
elif whichcolumn == 'genpop_withrecontact':
    mydata.to_csv('../data/CFA/data_genpop.csv')
    ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
    group = 'whichvisit'

ro.globalenv['group'] = group

# CFA
df_data = []    
for cln_ss, mdl in mdl_strs_new_new.items():
    do_metric = False
    do_scalar = False
    do_strict = False
    print("\n\n\n================================")
    print("================================")
    print(cln_ss.upper())
    print(mdl)
    print("================================")
    print("================================")
    # ====== CONFIG =====        
    ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new_new[cln_ss]
    fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
    ro.globalenv['fit_config'] = fit_config        
    print("----------------")
    out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
    config_p = extract_p(out_config)
    if float(config_p) >= 0.05:
        print('CONFIG INVARIANT chisq p = ' + str(config_p))
        do_metric = True
    else:
        (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
        if passed_flag:
            print('CONFIG chisq p = ' + str(config_p))
            print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
            do_metric = True
        else:
            print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
    if do_metric:

        # ====== METRIC =====
        fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
        ro.globalenv['fit_metric'] = fit_metric
        out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
        metric_p = extract_p(out_metric)
        if float(metric_p) >= 0.05:
            print('METRIC INVARIANT chisq p = ' + str(metric_p))
            do_scalar = True
        else:
            # significant:
            print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
            _ = print_problematic_items(out_metric)
            
        if do_scalar:
            # ====== SCALAR =====
            ro.globalenv['out_metric'] = out_metric
            fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
            ro.globalenv['fit_scalar'] = fit_scalar
            out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
            scalar_p = extract_p(out_scalar)
            if float(scalar_p) >= 0.05:
                print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                do_strict = True
            else:
                # significant:
                print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                _ = print_problematic_items(out_scalar)

            if do_strict:

                # ====== STRICT =====
                ro.globalenv['out_scalar'] = out_scalar
                fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                ro.globalenv['fit_strict'] = fit_strict
                out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                strict_p = extract_p(out_strict)
                if float(strict_p) >= 0.05:
                    print('STRICT INVARIANT chisq p = ' + str(strict_p))
                else:
                    # significant:
                    print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                    _ = print_problematic_items(out_strict)

            else:
                print('STRICT N/A')
                strict_p = 'NA'
        else:
            print('SCALAR N/A')
            print('STRICT N/A')
            scalar_p = 'NA'
            strict_p = 'NA'
    else:
        print('METRIC N/A')
        print('SCALAR N/A')
        print('STRICT N/A')
        metric_p = 'NA'
        scalar_p = 'NA'
        strict_p = 'NA'
    df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
df = pd.DataFrame(df_data)

print('DONE')

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.







HITOP_SUM
hitop_SUM =~anhedonic_depression + anxious_worry + appetite_gain + appetite_loss + cognitive_problems + hyposomnia + indecisiveness + insomnia + panic + separation_insecurity + shame_guilt + situational_phobia + social_anxiety
----------------


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG INVARIANT chisq p = 1


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

CONFIG INVARIANT chisq p = 0.949


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




METRIC INVARIANT chisq p = 0.449


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR INVARIANT chisq p = 0.915
STRICT INVARIANT chisq p = 0.528
DONE


In [ ]:
# ABLATIONS

mydata = mydata_forcfa
whichcolumn = 'enriched_withrecontact'

assert(whichcolumn in ['enriched_withrecontact', 'genpop_withrecontact'])
num_iter = 1000
rprint = ro.r('print')
summary = ro.r('summary')
ro.r('set.seed(12345)')
rngkind = "L'Ecuyer-CMRG"

mdl_strs_new_new = {'baars_hyperactivity_SUM': 'baars_hyperactivity_SUM =~hyperactivity_1 + hyperactivity_2 + hyperactivity_3 + hyperactivity_5'}

if whichcolumn == 'enriched_withrecontact':
    mydata.to_csv('../data/CFA/data_enriched.csv')
    ro.r("rdata <- read.csv('../data/CFA/data_enriched.csv', header=TRUE)") 
    group = 'whichvisit'
elif whichcolumn == 'genpop_withrecontact':
    mydata.to_csv('../data/CFA/data_genpop.csv')
    ro.r("rdata <- read.csv('../data/CFA/data_genpop.csv', header=TRUE)")
    group = 'whichvisit'

ro.globalenv['group'] = group

# CFA
df_data = []    
for cln_ss, mdl in mdl_strs_new_new.items():
    do_metric = False
    do_scalar = False
    do_strict = False
    print("\n\n\n================================")
    print("================================")
    print(cln_ss.upper())
    print(mdl)
    print("================================")
    print("================================")
    # ====== CONFIG =====        
    ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new_new[cln_ss]
    fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
    ro.globalenv['fit_config'] = fit_config        
    print("----------------")
    out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                parallelType="multicore", ncpus=4)
    config_p = extract_p(out_config)
    if float(config_p) >= 0.05:
        print('CONFIG INVARIANT chisq p = ' + str(config_p))
        do_metric = True
    else:
        (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
        if passed_flag:
            print('CONFIG chisq p = ' + str(config_p))
            print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
            do_metric = True
        else:
            print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
    if do_metric:

        # ====== METRIC =====
        fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
        ro.globalenv['fit_metric'] = fit_metric
        out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_config, 
                                                con=fit_metric, 
                                                param="loadings",
                                                parallelType="multicore", ncpus=4)
        metric_p = extract_p(out_metric)
        if float(metric_p) >= 0.05:
            print('METRIC INVARIANT chisq p = ' + str(metric_p))
            do_scalar = True
        else:
            # significant:
            print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
            _ = print_problematic_items(out_metric)
            
        if do_scalar:
            # ====== SCALAR =====
            ro.globalenv['out_metric'] = out_metric
            fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
            ro.globalenv['fit_scalar'] = fit_scalar
            out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                uncon=fit_metric, 
                                                con=fit_scalar, 
                                                param=ro.StrVector(["loadings","intercepts"]),
                                                parallelType="multicore", 
                                                ncpus=4)
            scalar_p = extract_p(out_scalar)
            if float(scalar_p) >= 0.05:
                print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                do_strict = True
            else:
                # significant:
                print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                _ = print_problematic_items(out_scalar)

            if do_strict:

                # ====== STRICT =====
                ro.globalenv['out_scalar'] = out_scalar
                fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                ro.globalenv['fit_strict'] = fit_strict
                out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                        uncon=fit_scalar, 
                                                        con=fit_strict, 
                                                        param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                        parallelType="multicore", 
                                                        ncpus=4)
                strict_p = extract_p(out_strict)
                if float(strict_p) >= 0.05:
                    print('STRICT INVARIANT chisq p = ' + str(strict_p))
                else:
                    # significant:
                    print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                    _ = print_problematic_items(out_strict)

            else:
                print('STRICT N/A')
                strict_p = 'NA'
        else:
            print('SCALAR N/A')
            print('STRICT N/A')
            scalar_p = 'NA'
            strict_p = 'NA'
    else:
        print('METRIC N/A')
        print('SCALAR N/A')
        print('STRICT N/A')
        metric_p = 'NA'
        scalar_p = 'NA'
        strict_p = 'NA'
    df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
df = pd.DataFrame(df_data)

print('DONE')

# GENPOP VS ENRICHED

In [65]:
data_path_genpop = '../data/mydata_1general_forCFA_removedGRIDchecks.csv'
data_path_enriched = '../data/mydata_1highrisk_forCFA_removedGRIDchecks.csv'
path_to_save = '../results/CFA_genpopenriched_removedGRIDchecks.csv'

mydata_genpop = pd.read_csv(data_path_genpop)
mydata_genpop = mydata_genpop.rename(columns={'Unnamed: 0': 'Subject'})

mydata_enriched = pd.read_csv(data_path_enriched)
mydata_enriched = mydata_enriched.rename(columns={'Unnamed: 0': 'Subject'})

In [66]:
mydata_enriched.shape

(255, 137)

In [67]:
mydata_enriched.head()

,Subject,hitop_sum,hitop_sum_recontact,baars_sum,baars_sum_recontact,phq_sum,phq_sum_recontact,gad_sum,gad_sum_recontact,mood_bothered,mood_bothered_recontact,anxiety_bothered,anxiety_bothered_recontact,attention_bothered,attention_bothered_recontact,moodanxiety_bothered,moodanxiety_bothered_recontact,inattention_sum,inattention_sum_recontact,hyperactivity_sum,hyperactivity_sum_recontact,impulsivity_sum,impulsivity_sum_recontact,sct_sum,sct_sum_recontact,anhedonic_depression,anhedonic_depression_recontact,anxious_worry,anxious_worry_recontact,appetite_gain,appetite_gain_recontact,appetite_loss,appetite_loss_recontact,cognitive_problems,cognitive_problems_recontact,hyposomnia,hyposomnia_recontact,indecisiveness,indecisiveness_recontact,insomnia,insomnia_recontact,panic,panic_recontact,separation_insecurity,separation_insecurity_recontact,shame_guilt,shame_guilt_recontact,situational_phobia,situational_phobia_recontact,social_anxiety,social_anxiety_recontact,well_being,well_being_recontact,inattention_1,inattention_2,inattention_3,inattention_4,inattention_5,inattention_6,inattention_7,inattention_8,inattention_9,hyperactivity_1,hyperactivity_2,hyperactivity_3,hyperactivity_4,hyperactivity_5,impulsivity_1,impulsivity_2,impulsivity_3,impulsivity_4,inattention_1_recontact,inattention_2_recontact,inattention_3_recontact,inattention_4_recontact,inattention_5_recontact,inattention_6_recontact,inattention_7_recontact,inattention_8_recontact,inattention_9_recontact,hyperactivity_1_recontact,hyperactivity_2_recontact,hyperactivity_3_recontact,hyperactivity_4_recontact,hyperactivity_5_recontact,impulsivity_1_recontact,impulsivity_2_recontact,impulsivity_3_recontact,impulsivity_4_recontact,gad_1,gad_2,gad_3,gad_4,gad_5,gad_6,gad_7,phq_1,phq_2,phq_3,phq_4,phq_5,phq_6,phq_7,phq_8,sct_1,sct_2,sct_3,sct_4,sct_5,sct_6,sct_7,sct_8,sct_9,gad_1_recontact,gad_2_recontact,gad_3_recontact,gad_4_recontact,gad_5_recontact,gad_6_recontact,gad_7_recontact,phq_1_recontact,phq_2_recontact,phq_3_recontact,phq_4_recontact,phq_5_recontact,phq_6_recontact,phq_7_recontact,phq_8_recontact,sct_1_recontact,sct_2_recontact,sct_3_recontact,sct_4_recontact,sct_5_recontact,sct_6_recontact,sct_7_recontact,sct_8_recontact,sct_9_recontact
0,0,80,115,13,19,14,17,7,16,True,True,False,True,False,False,True,True,6,13,6,6,1,0,7,10,11,22,6,14,9,9,0,0,1,6,0,0,0,0,12,12,3,5,10,13,9,10,7,9,12,15,11,10,0,0,0,0,1,0,3,1,1,2,0,1,0,3,1,0,0,0,1,0,0,1,2,2,3,1,3,0,0,1,3,2,0,0,0,0,1,0,0,3,1,2,0,1,1,3,3,3,2,1,0,0,2,0,2,0,2,1,0,0,3,1,3,3,0,3,3,2,3,3,3,3,3,0,0,0,0,1,3,0,3,3,0,0
1,2,123,133,41,43,15,14,14,18,True,True,True,True,True,True,True,True,21,22,11,13,9,8,25,21,16,16,17,19,2,0,2,2,12,12,9,6,9,9,5,7,8,10,12,14,10,11,2,4,19,23,8,7,3,3,2,3,1,1,3,3,2,1,3,3,1,3,2,2,3,2,3,3,2,2,2,2,2,3,3,2,2,3,3,3,2,2,2,2,2,2,2,2,2,3,1,2,2,2,2,1,2,3,1,3,3,3,3,3,3,3,1,3,3,2,2,3,3,3,2,1,2,2,1,1,2,3,2,3,3,3,3,3,1,1,1,3
2,3,84,84,0,12,12,9,16,10,False,True,False,True,False,False,False,True,0,2,0,2,0,8,12,7,12,14,10,9,3,3,2,2,4,3,0,0,4,3,12,12,12,8,4,9,9,4,0,3,12,14,14,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,3,1,3,1,3,1,3,3,1,2,3,1,2,3,3,2,0,1,0,1,1,1,1,1,3,3,0,1,1,1,3,3,0,2,0,1,1,3,3,1,0,0,0,1,0,0,0,0,3,3,0,0
3,5,156,129,15,16,21,21,21,15,True,True,True,True,True,True,True,True,12,15,3,1,0,0,20,17,27,17,20,18,2,0,6,4,10,9,0,0,9,8,12,11,12,10,20,17,11,12,1,1,26,22,6,5,1,1,1,1,1,2,1,2,2,1,0,1,1,0,0,0,0,0,1,2,2,1,1,3,1,2,2,1,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,2,3,3,3,3,3,3,1,3,3,2,0,3,3,1,2,3,1,2,2,3,2,2,3,3,3,3,3,2,3,3,1,3,2,1,0,3,2,2,1,3
4,8,96,91,8,6,7,6,15,14,False,False,True,True,False,False,True,True,5,4,1,1,2,1,11,8,8,11,18,16,2,2,0,0,5,5,0,0,5,3,3,4,5,5,15,13,6,4,1,0,28,28,18,21,0,1,0,1,1,0,0,1,1,1,0,0,0,0,0,2,0,0,0,1,0,0,1,0,0,2,0,1,0,0,0,0,0,1,0,0,3,3,3,3,0,0,3,0,1,2,2,0,2,0,0,1,0,0,0,0,2,3,2,3,3,3,3,3,0,0,2,0,1,2,2,0,1,0,0,1,1,0,0,0,1,2,2,1


## --> ANALYSIS OF FULL SETS
## (without differentiating between initial and recontact)

In [72]:
mydata_genpop['genpop_or_enriched'] = 'genpop'
mydata_enriched['genpop_or_enriched'] = 'enriched'
mydata_genpopenriched_full = pd.concat([mydata_genpop, mydata_enriched])

## --> analysis of initial and recontact separately

In [56]:
mydata_prepped_genpop = prep_data_recontactanalysis(mydata_genpop)
mydata_prepped_enriched = prep_data_recontactanalysis(mydata_enriched)

In [82]:
mydata_prepped_genpop['genpop_or_enriched'] = 'genpop'
mydata_prepped_genpop_initialonly = mydata_prepped_genpop[mydata_prepped_genpop.whichvisit == 'initialvisit']
mydata_prepped_genpop_recontactonly = mydata_prepped_genpop[mydata_prepped_genpop.whichvisit == 'recontactvisit']

mydata_prepped_enriched['genpop_or_enriched'] = 'enriched'
mydata_prepped_enriched_initialonly = mydata_prepped_enriched[mydata_prepped_enriched.whichvisit == 'initialvisit']
mydata_prepped_enriched_recontactonly = mydata_prepped_enriched[mydata_prepped_enriched.whichvisit == 'recontactvisit']

In [ ]:
mydata_genpopenriched_initialonly = pd.concat([mydata_genpop_initialonly, mydata_enriched_initialonly])
mydata_genpopenriched_recontactonly = pd.concat([mydata_genpop_recontactonly, mydata_enriched_recontactonly])

In [83]:
# cfa function

In [86]:
def run_cfa_genpopvsenriched(mydata_forcfa):
    mydata = mydata_forcfa
    
    num_iter = 1000
    rprint = ro.r('print')
    summary = ro.r('summary')
    ro.r('set.seed(12345)')
    rngkind = "L'Ecuyer-CMRG"

    mdl_strs_new_new = {'hitop_SUM': 'hitop_SUM =~anhedonic_depression + anxious_worry + appetite_gain + appetite_loss + cognitive_problems + hyposomnia + indecisiveness + insomnia + panic + separation_insecurity + shame_guilt + situational_phobia + social_anxiety', 
    'baars_SUM': 'baars_SUM =~inattention_sum + hyperactivity_sum + impulsivity_sum + sct_sum',
    'phq_SUM': 'phq_SUM =~phq_1 + phq_2 + phq_3 + phq_4 + phq_5 + phq_6 + phq_7 + phq_8',
    'gad_SUM': 'gad_SUM =~gad_1 + gad_2 + gad_3 + gad_4 + gad_5 + gad_6 + gad_7',
    'baars_inattention_SUM': 'baars_inattention_SUM =~inattention_1 + inattention_2 + inattention_3 + inattention_4 + inattention_5 + inattention_6 + inattention_7 + inattention_8 + inattention_9',
    'baars_hyperactivity_SUM': 'baars_hyperactivity_SUM =~hyperactivity_1 + hyperactivity_2 + hyperactivity_3 + hyperactivity_4 + hyperactivity_5',
    'baars_impulsivity_SUM': 'baars_impulsivity_SUM =~impulsivity_1 + impulsivity_2 + impulsivity_3 + impulsivity_4',
    'baars_sct_SUM': 'baars_sct_SUM =~sct_1 + sct_2 + sct_3 + sct_4 + sct_5 + sct_6 + sct_7 + sct_8 + sct_9'}

    mydata.to_csv('../data/CFA/data_genpopvsenriched.csv')
    ro.r("rdata <- read.csv('../data/CFA/data_genpopvsenriched.csv', header=TRUE)") 
    group = 'genpop_or_enriched'

    ro.globalenv['group'] = group
    
    # CFA
    df_data = [] 
    for cln_ss, mdl in mdl_strs_new_new.items():
        do_metric = False
        do_scalar = False
        do_strict = False
        print("\n")
        print(cln_ss.upper())
        print(mdl)
        print("\n")
        # ====== CONFIG =====        
        ro.globalenv[cln_ss+'_mdl'] = mdl_strs_new_new[cln_ss]
        fit_config = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV")')
        ro.globalenv['fit_config'] = fit_config        
        out_config = semtools.permuteMeasEq(nPermute=num_iter, # 100 for now, 1000 for the "real" run
                                                    con=fit_config, # In the case of testing configural invariance when modelType = "mgcfa", con is the configural model (implicitly, the unconstrained model is the saturated model, so use the defaults uncon = NULL and param = NULL).
                                                    parallelType="multicore", ncpus=4)
        config_p = extract_p(out_config)
        if float(config_p) >= 0.05:
            print('CONFIG INVARIANT chisq p = ' + str(config_p))
            do_metric = True
        else:
            (passed_flag, my_cfi, my_tli, my_rmsea) = check_secondary_criteria(fit_config)
            if passed_flag:
                print('CONFIG chisq p = ' + str(config_p))
                print('PASSED SECONDARY CRITERIA WITH CFI = ' + str(my_cfi) + ' TLI = ' + str(my_tli) + ' RMSEA = ' + str(my_rmsea))
                do_metric = True
            else:
                print('CONFIG INVARIANT NOT PASSED, chisq p = ' + str(config_p))
       
        if do_metric:
    
            # ====== METRIC =====
            fit_metric = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal="loadings")')
            ro.globalenv['fit_metric'] = fit_metric
            out_metric = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_config, 
                                                    con=fit_metric, 
                                                    param="loadings",
                                                    parallelType="multicore", ncpus=4)
            metric_p = extract_p(out_metric)
            if float(metric_p) >= 0.05:
                print('METRIC INVARIANT chisq p = ' + str(metric_p))
                do_scalar = True
            else:
                # significant:
                print('METRIC INVARIANT NOT PASSED, chisq p = ' + str(metric_p))
                _ = print_problematic_items(out_metric)
                
            if do_scalar:
                # ====== SCALAR =====
                ro.globalenv['out_metric'] = out_metric
                fit_scalar = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts"))')
                ro.globalenv['fit_scalar'] = fit_scalar
                out_scalar = semtools.permuteMeasEq(nPermute=num_iter, 
                                                    uncon=fit_metric, 
                                                    con=fit_scalar, 
                                                    param=ro.StrVector(["loadings","intercepts"]),
                                                    parallelType="multicore", 
                                                    ncpus=4)
                scalar_p = extract_p(out_scalar)
                if float(scalar_p) >= 0.05:
                    print('SCALAR INVARIANT chisq p = ' + str(scalar_p))
                    do_strict = True
                else:
                    # significant:
                    print('SCALAR INVARIANT NOT PASSED, chisq p = ' + str(scalar_p))
                    _ = print_problematic_items(out_scalar)
    
                if do_strict:
    
                    # ====== STRICT =====
                    ro.globalenv['out_scalar'] = out_scalar
                    fit_strict = ro.r(f'cfa({cln_ss}_mdl, data = rdata, group = group, estimator = "WLSMV", group.equal=c("loadings", "intercepts", "residuals"))')
                    ro.globalenv['fit_strict'] = fit_strict
                    out_strict = semtools.permuteMeasEq(nPermute=num_iter, 
                                                            uncon=fit_scalar, 
                                                            con=fit_strict, 
                                                            param=ro.StrVector(["loadings","intercepts", "residuals"]),
                                                            parallelType="multicore", 
                                                            ncpus=4)
                    strict_p = extract_p(out_strict)
                    if float(strict_p) >= 0.05:
                        print('STRICT INVARIANT chisq p = ' + str(strict_p))
                    else:
                        # significant:
                        print('STRICT INVARIANT NOT PASSED, chisq p = ' + str(strict_p))
                        _ = print_problematic_items(out_strict)
    
                else:
                    print('STRICT N/A')
                    strict_p = 'NA'
            else:
                print('SCALAR N/A')
                print('STRICT N/A')
                scalar_p = 'NA'
                strict_p = 'NA'
        else:
            print('METRIC N/A')
            print('SCALAR N/A')
            print('STRICT N/A')
            metric_p = 'NA'
            scalar_p = 'NA'
            strict_p = 'NA'
        df_data.append({cln_ss: [config_p, metric_p, scalar_p, strict_p]})
    df = pd.DataFrame(df_data)
    
    print('DONE')

In [ ]:
# data available:

#mydata_genpopenriched_full
#mydata_genpopenriched_initialonly
#mydata_genpopenriched_recontactonly

In [ ]:
run_cfa_genpopvsenriched(mydata_genpopenriched_full)

R[write to console]: No AFIs were selected, so only chi-squared will be permuted.






HITOP_SUM
hitop_SUM =~anhedonic_depression + anxious_worry + appetite_gain + appetite_loss + cognitive_problems + hyposomnia + indecisiveness + insomnia + panic + separation_insecurity + shame_guilt + situational_phobia + social_anxiety




R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0.001
PASSED SECONDARY CRITERIA WITH CFI = 0.995 TLI = 0.994 RMSEA = 0.055
METRIC INVARIANT chisq p = 0.204


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




...

CONFIG chisq p = 0
PASSED SECONDARY CRITERIA WITH CFI = 0.999 TLI = 0.998 RMSEA = 0.053
METRIC INVARIANT NOT PASSED, chisq p = 0.009
Problematic items:
[1] "Parameter 'gad_SUM=~gad_6' may differ between Groups 'genpop' and 'enriched'."


R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




SCALAR N/A
STRICT N/A


BAARS_INATTENTION_SUM
baars_inattention_SUM =~inattention_1 + inattention_2 + inattention_3 + inattention_4 + inattention_5 + inattention_6 + inattention_7 + inattention_8 + inattention_9




R[write to console]: No AFIs were selected, so only chi-squared will be permuted.




CONFIG chisq p = 0
PASSED SECONDARY CRITERIA WITH CFI = 0.992 TLI = 0.99 RMSEA = 0.041


In [ ]:
run_cfa_genpopvsenriched(mydata_initialonly_full)

In [ ]:
run_cfa_genpopvsenriched(mydata_recontactonly_full)